<a href="https://colab.research.google.com/github/aagoldberg/farcaster-social-experiments/blob/main/botDetector_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# repliesWithHidden_final.csv

In [2]:
# Step 1: Install Necessary Libraries
!pip install transformers torch pandas scikit-learn

In [ ]:
# Step 2: Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader, Dataset

In [ ]:
from google.colab import auth
# from google.cloud import bigquery
from google.colab import data_table

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 3: Load and Prepare the Data

# Load the CSV data
df = pd.read_csv(f'/content/drive/My Drive/repliesWithHidden_final.csv')

In [ ]:
df.head()

,hash,thread_hash,author_fid,author_username,author_displayName,author_followerCount,author_followingCount,text,timestamp,replies_count,reactions_count,recasts_count,view_count,quote_count,warps_tipped,hidden
0,0x5bb632a9b61fda7de7a74d667d5d254353549a41,0x5bb632a9b61fda7de7a74d667d5d254353549a41,1941,aleph,Aleph 🎩 ↑,5930,711,I tipped 8869 DEGEN to 2 accounts yesterday. C...,1723104009000,0,3,0,NaN,0.0,NaN,False
1,0x3ff3a9fa56545df247369327046a00b6cfbc976b,0x5bb632a9b61fda7de7a74d667d5d254353549a41,510012,animeworld,Otaku 🎩 🔵 Ⓜ️ 🎭,365,1062,"Whoa, that's awesome! 🤯 Did you get yours? 🚀",1723104415000,0,0,0,NaN,NaN,NaN,True
2,0x910779cfd46dc2c155ba4f771d999db5206e7ae7,0x5bb632a9b61fda7de7a74d667d5d254353549a41,814082,nixor,Nixor🎭🎩,96,157,Nice one!,1723104238000,0,0,0,NaN,NaN,NaN,True
3,0x8958f882b10b4842a5d798483296eaae0d35fa46,0x8958f882b10b4842a5d798483296eaae0d35fa46,1941,aleph,Aleph 🎩 ↑,5930,711,Wisdom. Too real. https://x.com/elonmusk/statu...,1723049550000,2,1,0,NaN,0.0,NaN,False
4,0x314037988f042958979ec5707b6ffedc1789f4dd,0x8958f882b10b4842a5d798483296eaae0d35fa46,621286,gamemaker0,Gamemaker.eth 🍖,269,246,Big truth 🧠,1723094611000,0,0,0,NaN,NaN,NaN,True


In [ ]:
# Sample data preparation (simplified for demonstration)
df['is_bot'] = df['hidden'].apply(lambda x: 1 if x is True else 0)  # Example bot detection label

# Split into features and target
X = df['text']
y = df['is_bot']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.fillna('[NO TEXT]').astype(str)
X_val = X_val.fillna('[NO TEXT]').astype(str)



In [ ]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=128)

# Convert labels to lists
train_labels = y_train.tolist()
val_labels = y_val.tolist()


In [ ]:
class BotDetectionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)



In [ ]:
train_dataset = BotDetectionDataset(train_encodings, train_labels)
val_dataset = BotDetectionDataset(val_encodings, val_labels)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

eval_results = trainer.evaluate()

print(f"Validation loss: {eval_results['eval_loss']}")
print(f"Validation accuracy: {eval_results['eval_accuracy']}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.286500,0.303358


Epoch,Training Loss,Validation Loss
1,0.286500,0.303358
2,0.248900,0.276675
3,0.181200,0.242819


Validation loss: 0.2428194284439087


KeyError: 'eval_accuracy'

In [ ]:
eval_results

{'eval_loss': 0.2428194284439087,
 'eval_runtime': 126.1921,
 'eval_samples_per_second': 3.122,
 'eval_steps_per_second': 0.198,
 'epoch': 3.0}

In [ ]:
new_text = ["Revolutionary solution for cross-chain payments. Easy, seamless transactions with any token.", "Exciting news for Ethereum investors."]

# Tokenize the new text
new_encodings = tokenizer(new_text, truncation=True, padding=True, max_length=128, return_tensors='pt')

# The input_ids and attention_mask are needed for the model
input_ids = new_encodings['input_ids']
attention_mask = new_encodings['attention_mask']

In [ ]:
# Make sure the model is in evaluation mode
model.eval()

# Disable gradient calculation for inference
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    probabilities = torch.sigmoid(logits)
    predictions = (probabilities > 0.5).long()  # Convert probabilities to 0 or 1

# Convert predictions to a list and print
predicted_labels = predictions.tolist()
for text, label in zip(new_text, predicted_labels):
    print(f"Text: {text}\nPredicted Label: {'Bot' if label == 1 else 'Not a Bot'}\n")


Text: Revolutionary solution for cross-chain payments. Easy, seamless transactions with any token.
Predicted Label: Not a Bot

Text: Exciting news for Ethereum investors.
Predicted Label: Not a Bot



In [ ]:
probabilities

tensor([[0.4225, 0.4298],
        [0.1921, 0.7072]])

In [1]:
eval_results

NameError: name 'eval_results' is not defined